# MNIST Training using PyTorch

参考：wandb

https://github.com/wandb/examples/blob/master/examples/pytorch/pytorch-mnist-sagemaker/pytorch_mnist.ipynb

SageMaker sample

https://github.com/aws/amazon-sagemaker-examples/blob/main/hyperparameter_tuning/pytorch_mnist/hpo_pytorch_mnist.ipynb

In [1]:
!pip install wandb -qqq

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)

---

## Background

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using PyTorch.

MNISTは、手書きの数字を分類するために広く使われているデータセットである。このデータセットは、手書き数字の28x28ピクセルのグレースケール画像70,000枚から構成される。このデータセットは60,000枚の学習画像と10,000枚のテスト画像に分割される。10個のクラスがあります（10桁の数字にそれぞれ1つずつ）。このチュートリアルでは、PyTorchを使ってSageMaker上でMNISTモデルを学習・テストする方法を紹介します。


For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

Let's start by creating a SageMaker session and specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).


In [2]:
#!pip install -U sagemaker==2.87.0

In [3]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/DEMO-pytorch-mnist-wandb2'

role = sagemaker.get_execution_role()

In [4]:
sagemaker.__version__

'2.87.0'

## Data
### Getting the data



In [5]:
#!pip install ipywidgets widgetsnbextension

In [6]:
from torchvision.datasets import MNIST
from torchvision import transforms

MNIST.mirrors = ["https://sagemaker-sample-files.s3.amazonaws.com/datasets/image/MNIST/"]

MNIST(
    'data',
    download=True,
    transform=transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )
)

/opt/conda/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

### Uploading the data to S3
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.


In [7]:
inputs = sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)
print('input spec (in this case, just an S3 path): {}'.format(inputs))

input spec (in this case, just an S3 path): s3://sagemaker-us-east-1-805433377179/sagemaker/DEMO-pytorch-mnist-wandb2


In [8]:
inputs

's3://sagemaker-us-east-1-805433377179/sagemaker/DEMO-pytorch-mnist-wandb2'

## Train
### Training script
The `mnist.py` script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.
* `SM_NUM_GPUS`: The number of gpus available in the current container.
* `SM_CURRENT_HOST`: The name of the current container on the container network.
* `SM_HOSTS`: JSON encoded list containing all the hosts .

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (``if __name__=='__main__':``) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

For example, the script run by this notebook:

In [13]:
!pygmentize src/mnist.py

import argparse
import json
import logging
import os
import sys
import wandb

#import sagemaker_containers
import torch
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
import torch.utils.data.distributed
from torchvision import datasets, transforms

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))


# Based on https://github.com/pytorch/examples/blob/master/mnist/main.py
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = 

### Run training in SageMaker

The `PyTorch` class allows us to run our training function as a training job on SageMaker infrastructure. We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters. In this case we are going to run our training job on 2 ```ml.c4.xlarge``` instances. But this example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)). The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the `mnist.py` script above.


In [17]:
import wandb
wandb.login()
settings = wandb.setup().settings
current_api_key = wandb.wandb_lib.apikey.api_key(settings=settings)

In [18]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point='mnist.py',
                    source_dir="src",
                    role=role,
                    py_version='py3',
                    framework_version='1.8.0',
                    instance_count=1,
                    instance_type='ml.c5.2xlarge',
                    hyperparameters={
                        'epochs': 1,
                        'backend': 'gloo'
                    },
                   environment={"WANDB_API_KEY": current_api_key})

After we've constructed our `PyTorch` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem, so our training script can simply read the data from disk.


In [19]:
estimator.fit({'training': inputs})

2022-05-30 08:51:26 Starting - Starting the training job...
2022-05-30 08:51:50 Starting - Preparing the instances for trainingProfilerReport-1653900686: InProgress
......
2022-05-30 08:52:51 Downloading - Downloading input data...
2022-05-30 08:53:16 Training - Downloading the training image...
2022-05-30 08:53:51 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-05-30 08:53:43,505 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-05-30 08:53:43,507 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-05-30 08:53:43,516 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-05-30 08:53:43,522 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-05-30 08:53:43,930 sagemaker-training-toolkit INFO     Installi

UnexpectedStatusException: Error for Training job pytorch-training-2022-05-30-08-51-26-294: Failed. Reason: AlgorithmError: ExecuteUserScriptError:
Command "/opt/conda/bin/python3.6 mnist.py --backend gloo --epochs 1"
wandb: Currently logged in as: yito. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.12.17
wandb: Run data is saved locally in /opt/ml/code/wandb/run-20220530_085351-pytorch-training-2022-05-30-08-51-26-294-algo-1
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run pytorch-training-2022-05-30-08-51-26-294-algo-1
wandb: â­ï¸ View project at https://wandb.ai/yito/sm-pytorch-mnist-new3
wandb: ð View run at https://wandb.ai/yito/sm-pytorch-mnist-new3/runs/pytorch-training-2022-05-30-08-51-26-294-algo-1
Traceback (most recent call last):
  File "mnist.py", line 284, in <module>
    train(args)
  File "mnist.py", line 115, in train
    train_loader = _get_train_data_loader(args.batch_size, args.data_dir, is_distribut

## 複数ジョブ発行

In [ ]:
lrs = [0.01, 0.001, 0.05, 0.1, 0.2]
for lr in lrs:
    estimator = PyTorch(entry_point='mnist.py',
                        source_dir="src",
                        role=role,
                        py_version='py3',
                        framework_version='1.8.0',
                        instance_count=1,
                        instance_type='ml.c5.2xlarge',
                        hyperparameters={
                            'epochs': 10,
                            'backend': 'gloo',
                            'lr': lr,
                        },
                       environment={"WANDB_API_KEY": current_api_key})
    estimator.fit({'training': inputs}, wait=False)